## Linear Methods for Classification

This section deals with Logistic Regression and Linear Discriminant Analysis for classification problems.

### Logistic Regression
For a binary classification problem with class $0$ or $1$, one could apply regression by least squares. The $X \hat{\beta}$ so obtained in this case can be shown to actually be an estimate of $Pr(Y=1 \mid X)$. But the estimates from linear regression tend to be outside the [0, 1] interval, and hence not useful for a probabilistic interpretation. In Logistic regression, we avoid this problem by applying a function that gives outputs between $0$ to $1$ for all input values - the ___logit___ aka ___sigmoid___ function. 

Rather than modelling the response variable $Y$ directly, logistic regression models the probability that $Y$ belongs to a particular category, i.e., the conditional probability of $Y$ given the predictors $X$: $p(X) = P(Y=1 \mid X)$. These take values in the interval $[0, 1]$. We could then predict (or assign) an observation to a particular class based on a threshold probability value. For example, we could assign the observation to class $1$ if $p(X) = Pr(Y = 1 \mid X) > 0.5$, and to the other class otherwise. Depending on the application, we might also want to be conservative about our prediction and instead choose lower values like $0.1$ as threshold.

We model the $p(x)$ as a sigmoid function:

$$p(X) = \frac{\exp(\beta_0 + \beta_1X)}{1 + \exp(\beta_0 + \beta_1X)}$$

A bit of algebra would show that:

$$\log \frac{p(X)}{1 - p(X)} = \beta_0 + \beta_1X$$

$\frac{p(X)}{1 - p(X)}$ is called the ___odds___, and the ___log-odds___ (aka logit) is shown to be linear in $X$. For logistic regression, the optimization problem we want to solve is to find estimates for $\beta$ such that the estimated probability $\hat p(x_i)$ corresponds as closely as possible to the original training observation. This approach is known as the ___maximum likelihood___ approach. Formally, we seek to find estimates $\beta$ that maximizes the _likelihood_ function:

$$L(\beta) = \prod_{i:y_i=1}p(x_i) \prod_{i^\prime: y_{i^\prime} = 0}(1 - p(x_{i^\prime}))$$

or any monotonic function of $L$, specifically the ___log-likelihood___ as is often used for convenience:

\begin{align}
\ell(\beta)
&= \log L(\beta)\\
&= \sum_{i = 1}^{n} \{y_i \log p(x_i) + (1 - y_i)\log(1 - p(x_i))\}\\
&= \sum_{i = 1}^{n}\{y_i\beta^Tx_i - \log (1 + e^{\beta^Tx_i})\}
\end{align}

The coefficients estimated from logistic regression refer to the change in log-odds of the response and not the response itself.


### Linear Discriminant Analysis
In logistic regression, we directly modeled the conditional distribution $Pr(Y = k \mid X = x)$. In Linear Discriminant Analysis (LDA), we instead model the distribution of predictors $X$ in each class, and then use Bayes's Theorem to compute the estimates for $Pr(Y = k \mid X = x)$.

We have $K \ge 2$ classes in the response variable. Let $f_k(x)$ be the density function of $X$ in class $k$, and $\pi_k$ be the prior probability of class $k$,  i.e., the probability that a randomly chosen observation comes from the $k$th class, with $\sum_{k=1}^K \pi_k = 1$. Baye's theorem now gives us:

$$Pr(Y = k \mid X = x) = p_k(x) = \frac{\pi_kf_k(x)}{\sum_{l=1}^K \pi_l f_l(x)}$$

If we have estimates for $\pi_k$ and $f_k$, we can compute $p_k(x)$. Lets model $f_k(x)$ as a Gaussian:

$$f_k(x) = \frac{1}{(2 \pi)^{p / 2} \lvert\Sigma\rvert ^{1/2}} \exp \left( \frac{1}{2}(x - \mu)^T\Sigma^{-1}(x - \mu) \right)$$

where $p$ is the number of predictors, $\mu$ is the mean of $X$ and $\Sigma = Cov(X)$ with $\Sigma_k = \Sigma$ $\forall k$. In practice, we do not know the parameters of the Gaussian distributions, and hence have to be estimated:

\begin{align}
\hat \pi_k &= \frac{n_k}{n} \\
\hat \mu_k &= \frac{1}{n_k} \sum_{i:y_i=k}x_i \\
\hat \Sigma &= \frac{1}{n - K} \sum_{k=1}^K \sum_{i:y_i=k} (x_i - \hat \mu_k) (x_i - \hat \mu_k)^T
\end{align}

where $n_k$ is the number of class-$k$ observations. Once we have all this, we can calculate $p_k(x)$ and assign observation $X = x$ to the class for which $p_k(x)$ is largest. Some algebra will show that, this problem is equivalent to assigning the observation to the class for which the ___linear discriminant function___, $\delta_k$ is largest:

$$\delta_k(x) = x^T \Sigma^{-1} \mu_k - \frac{1}{2} \mu_k^T \Sigma^{-1} \mu_k + \log \pi_k $$

We can see that the discriminant function (and also log-odds) is linear in $X$. LDA makes a lot of assumptions about the distribution of the observations, and hence must be considered carefully before using. If the equality of covariances assumption is not met, one can consider using ___Quadratic Discriminant Analysis___ (QDA) which produces quadratic decision boundaries.